# Домашнее задание №2

Это задание состоит из двух сюжетов:

1) Workflow ML-модели. Несложный код, который вы пишете всегда - чтобы обучить модель от начала (загрузка данных) и до конца (оценка качества и улучшение модели).

2) Самостоятельная реализация различных способов кодирования категориальных признаков - полезное упражнение для понимания кодировок (в дальнейшем будем использовать готовые функции).

## <font color='green'>Часть 1. ML workflow (**всего 5 баллов**)</font>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы. 

In [ ]:
from sklearn.datasets import load_diabetes

data = load_diabetes(as_frame=True)
print(data.DESCR)
df = data.frame
df.head()

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


Будем решать задачу регрессии: необходимо предсказать уровень сахара в крови по характеристикам пациентов

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("target")

In [ ]:
X, y = np.array(df.drop("target", axis=1)), np.array(df["target"])

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [ ]:
model = LinearRegression().fit(X_train, y_train)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [ ]:
print("train MSE error", mean_squared_error(y_train, model.predict(X_train)))
print("test MSE error", mean_squared_error(y_test, model.predict(X_test)))

train MSE error 2924.0463790726394
test MSE error 2821.750981001311


In [ ]:
print("train r2 error", r2_score(y_train, model.predict(X_train)))
print("test r2 error", r2_score(y_test, model.predict(X_test)))

train r2 error 0.5244124363545944
test r2 error 0.4772897164322617


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [ ]:
from sklearn import svm

#clf = svm.SVC(kernel='linear', C=1, random_state=42)
model = LinearRegression()

scores = cross_val_score(model, X, y, cv=5)

In [ ]:
scores.mean()

0.48231643590864215

### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [ ]:
from sklearn.linear_model import Lasso
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

param_grid = {'alpha': [i/10 for i in range(1, 11)]}

reg = linear_model.Lasso()
#grid = GridSearchCV(reg, param_grid, scoring='neg_mean_squared_error', cv=5)
grid = GridSearchCV(reg, param_grid)
grid.fit(X_train, y_train)

grid.best_params_

{'alpha': 0.1}

### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [ ]:
best_estimator = grid.best_estimator_

best_score = grid.best_score_

print(best_estimator, best_score)

Lasso(alpha=0.1) 0.44654756885370095


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [ ]:
reg = linear_model.Lasso(alpha=0.1)

reg.fit(X_train, y_train)


print("train R2 error", r2_score(y_train, reg.predict(X_train)))
print("test R2 error", r2_score(y_test, reg.predict(X_test)))

train R2 error 0.5134140107713475
test R2 error 0.4859194402036221


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [ ]:
from sklearn.preprocessing import StandardScaler

X_poly = PolynomialFeatures(degree=2).fit_transform(X)
X = np.concatenate((X, X_poly), axis=1)

pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('linreg', LinearRegression())
    ])

# ('linreg', LinearRegression())

r2_scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')

# Print the mean r2-score and the standard deviation
print('R2-score:', r2_scores.mean())
print('Standard deviation:', r2_scores.std())


R2-score: -69.73307280869885
Standard deviation: 45.68653950395458


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [ ]:
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X, y, test_size=0.3, random_state=42)

pipeline.fit(X_train_poly, y_train_poly)

print("train r2_poly error", r2_score(y_train_poly, pipeline.predict(X_train_poly)))
print("test MSE_poly error", mean_squared_error(y_train_poly, pipeline.predict(X_train_poly)))

train r2_poly error 1.0
test MSE_poly error 1.0026030503572737e-22


In [ ]:
print("test r2_poly error", r2_score(y_test_poly, pipeline.predict(X_test_poly)))
print("test MSE_poly error", mean_squared_error(y_test_poly , pipeline.predict(X_test_poly)))

test r2_poly error -38.73443136292106
test MSE_poly error 214498.68924058002


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

1)Качество модели ниже среднего. Переобученной ее назвать сложно из-за малой разницы в предсказаниях на тестовой и обучающей выборке

2)L1 регуляризация не улучшила качество судя по предсказаниям с добавлением L1

3)Добавление полиномов ухудшило ситуацию. Модель стала сильно переобученной, тк качество на тестовой выборке ужасное, а на обучающей идеальное.

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы 
(**дополнительно 1 балл**)

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.2 MB/s eta 0:00:00


In [ ]:
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

train_dataset = cb.Pool(X_train, y_train) 
test_dataset = cb.Pool(X_test, y_test)

model = cb.CatBoostRegressor(loss_function='RMSE')

grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

model.grid_search(grid, train_dataset)

pred = model.predict(X_test)
mse = mean_squared_error(y_test, pred)
r2 = r2_score(y_test, pred)

Выходные данные были обрезаны до нескольких последних строк (5000).
156:	learn: 37.5210401	test: 63.1502131	best: 63.1502131 (156)	total: 585ms	remaining: 160ms
157:	learn: 37.3480616	test: 63.1722074	best: 63.1502131 (156)	total: 587ms	remaining: 156ms
158:	learn: 37.1754960	test: 63.1231008	best: 63.1231008 (158)	total: 589ms	remaining: 152ms
159:	learn: 37.0863952	test: 63.0798139	best: 63.0798139 (159)	total: 601ms	remaining: 150ms
160:	learn: 36.9962652	test: 63.0391999	best: 63.0391999 (160)	total: 603ms	remaining: 146ms
161:	learn: 36.9068558	test: 63.0047611	best: 63.0047611 (161)	total: 613ms	remaining: 144ms
162:	learn: 36.7499912	test: 62.9110283	best: 62.9110283 (162)	total: 616ms	remaining: 140ms
163:	learn: 36.6266417	test: 62.9190025	best: 62.9110283 (162)	total: 618ms	remaining: 136ms
164:	learn: 36.5167208	test: 62.9383115	best: 62.9110283 (162)	total: 620ms	remaining: 131ms
165:	learn: 36.4269836	test: 62.9636469	best: 62.9110283 (162)	total: 624ms	remaining: 128ms
16

Качество модели повысилось не очень существенно, но повысилось!

## <font color='green'>Часть 2. Target encoding (**всего 5 баллов**)</font>

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [ ]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [ ]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [ ]:
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [ ]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


#### Способ 2

In [ ]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621699


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False)

kf.get_n_splits(all_data)
for train_idx, val_idx in kf.split(all_data):
        train_data=all_data.iloc[train_idx]  
        val_data=all_data.iloc[val_idx]
    
        item_id_target_mean = train_data.groupby('item_id')['target'].mean()
    
        all_data['item_target_enc'].iloc[val_idx] = val_data['item_id'].map(item_id_target_mean)

all_data['item_target_enc'].fillna(0.3343, inplace=True)  

encoded_feature = all_data['item_target_enc'].values

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)



/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


0.4164590712798811


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [ ]:
sum_targets = all_data.groupby('item_id')['target'].transform('sum')
n_targets = all_data.groupby('item_id')['target'].transform('count')
all_data['item_target_enc']=(sum_targets-all_data['target'])/ (n_targets-1)

print(np.corrcoef(all_data['target'].values, all_data['item_target_enc'].values)[0][1])

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4803848311293002
0.4803848311293002


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [ ]:
globalmean = 0.3343
alpha = 100

sum_targets = all_data.groupby('item_id')['target'].transform('sum')
n_targets = all_data.groupby('item_id')['target'].transform('count')
mean_target = all_data.groupby('item_id')['target'].transform('mean')

all_data['item_target_enc']=(mean_target*n_targets + globalmean*alpha)/ (n_targets+alpha)

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4818198797097264


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [ ]:
sum = all_data.groupby('item_id')["target"].cumsum()-all_data["target"]
count = all_data.groupby('item_id')["target"].cumcount()

all_data['item_target_enc'] = sum/count

all_data['item_target_enc'].fillna(0.3343, inplace=True)
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.5025245211081697


Ожидаемый ответ 0.5025